In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('./gdrive/MyDrive/AEGAN/AE_GAN_JOC/APE-GAN')
!nvidia-smi
!ls

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import CarliniL2Method
from art.attacks.evasion import DeepFool
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10
from art.defences.trainer import AdversarialTrainer
from art.defences.trainer import AdversarialTrainerMadryPGD

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


In [ ]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_cifar10()
x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

In [ ]:
model = ResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
)

In [ ]:
classifier.fit(x_train, y_train, batch_size=128, nb_epochs=100)

In [ ]:
predictions1 = classifier.predict(x_test)
accuracy1 = np.sum(np.argmax(predictions1, axis=1) == np.argmax(y_test, axis=1))/len(y_test)
print('Accuracy1 test examples:{}%'.format(accuracy1*100))

In [ ]:
# attack_fgsm1 = FastGradientMethod(estimator=classifier,eps=0.1)
# attack_fgsm2 = FastGradientMethod(estimator=classifier,eps=0.3)
# attack_fgsm3 = FastGradientMethod(estimator=classifier,eps=0.5)
# attack_fgsm4 = FastGradientMethod(estimator=classifier,eps=0.7)
# fgsm_at = AdversarialTrainer(classifier, [attack_fgsm1,attack_fgsm2,attack_fgsm3,attack_fgsm4], ratio=0.5)
# fgsm_at.fit(x_train, y_train, batch_size=128, nb_epochs=100)

pgd_at = AdversarialTrainerMadryPGD(classifier, nb_epochs=300, batch_size=128, eps = 8, eps_step = 2, max_iter = 7)
pgd_at.fit(x_train, y_train)

In [ ]:
predictions2 = classifier.predict(x_test)
accuracy2 = np.sum(np.argmax(predictions2, axis=1) == np.argmax(y_test, axis=1))/len(y_test)
print('Accuracy2 on test examples:{}%'.format(accuracy2*100))

Accuracy2 on test examples:80.01%


In [ ]:
attack = FastGradientMethod(estimator=classifier,eps=0.01)
# attack = BasicIterativeMethod(estimator=classifier, eps=0.7)
# attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma=0.7)
# attack = CarliniL2Method(classifier=classifier, max_iter=500)
# attack = DeepFool(classifier=classifier)                                                
adv = attack.generate(x=x_test[0:2000])
# cw = torch.load('./CW2.tar')
# adv = cw['CW']
# y_test = cw['y']
predictions3 = classifier.predict(adv)
accuracy3 = np.sum(np.argmax(predictions3, axis=1) == np.argmax(y_test[0:2000], axis=1))/len(y_test[0:2000])
# accuracy3 = np.sum(np.argmax(predictions3, axis=1) == y_test.numpy())/len(y_test)
print('Accuracy3 on test examples:{}%'.format(accuracy3*100))